In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("./raw_data/data.csv", encoding="ISO-8859-1")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [16]:
# DATA CLEANING & TRANSFORMATION

raw_data_length = len(df)

# ---------------------------- Drop nan ------------------------------------------------------
print(f"The dataset contains {df.isnull().sum().sum()} duplicate rows that need to be removed.")
df = df.dropna(subset=["Description", "CustomerID"])
assert df.isnull().sum().sum() == 0
# --------------------------------------------------------------------------------------------
# ---------------------------- Drop duplciates -----------------------------------------------

print(f"The dataset contains {df.duplicated().sum()} duplicate rows that need to be removed.")
df.drop_duplicates(inplace=True)
assert df.duplicated().sum() == 0
# --------------------------------------------------------------------------------------------

# --------------------------- Canceled transaction cut ---------------------------------------
df["Transaction_Status"] = np.where(df["InvoiceNo"].astype(str).str.startswith("C"), "Cancelled", "Completed").copy()
df = df[df["Transaction_Status"] == "Completed"]
# --------------------------------------------------------------------------------------------

# -------------------------- delete anomalous stock codes ------------------------------------
unique_stock_codes = df['StockCode'].unique()
numeric_char_counts_in_unique_codes = pd.Series(unique_stock_codes).apply(lambda x: sum(c.isdigit() for c in str(x))).value_counts()
anomalous_stock_codes = [code for code in unique_stock_codes if sum(c.isdigit() for c in str(code)) in (0, 1)]
df = df[~df['StockCode'].isin(anomalous_stock_codes)]
# --------------------------------------------------------------------------------------------

# Resetting the index of the cleaned dataset
df.reset_index(drop=True, inplace=True)

# delete unitprice == 0
df = df[df['UnitPrice'] > 0]

print(f"raw data len: {raw_data_length}; cleaned data len: {len(df)}")

The dataset contains 136534 duplicate rows that need to be removed.
The dataset contains 5225 duplicate rows that need to be removed.
raw data len: 541909; cleaned data len: 391150


In [17]:
# FEATURE ENGINERING

"""
    Recency (R): This metric indicates how recently a customer has made a purchase.
    A lower recency value means the customer has purchased more recently, indicating higher engagement with the brand.

    Frequency (F): This metric signifies how often a customer makes a purchase within a certain period.
    A higher frequency value indicates a customer who interacts with the business more often, suggesting higher loyalty or satisfaction.

    Monetary (M): This metric represents the total amount of money a customer has spent over a certain period.
    Customers who have a higher monetary value have contributed more to the business, indicating their potential high lifetime value.
"""
# --------------------------------- RECENCY ----------------------------------------------------------
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["InvoiceDay"] = df["InvoiceDate"].dt.date

customer_data = df.groupby("CustomerID")["InvoiceDay"].max().reset_index()
most_recent_date = df["InvoiceDay"].max()

customer_data["InvoiceDay"] = pd.to_datetime(customer_data["InvoiceDay"])
most_recent_date = pd.to_datetime(most_recent_date)

customer_data["Days_Since_Last_Purchase"] = (most_recent_date - customer_data["InvoiceDay"]).dt.days
customer_data

customer_data.drop(columns=["InvoiceDay"], inplace=True)

# ------------------------------- FREQ --------------------------------------------------------------

total_transactions = df.groupby("CustomerID")["InvoiceNo"].nunique().reset_index()
total_transactions.rename(columns={"InvoiceNo": "Total_Transactions"}, inplace=True)

total_products_purchased = df.groupby("CustomerID")["Quantity"].sum().reset_index()
total_products_purchased.rename(columns={"Quantity": "Total_Products_Purchased"}, inplace=True)

customer_data = pd.merge(customer_data, total_transactions, on="CustomerID")
customer_data = pd.merge(customer_data, total_products_purchased, on="CustomerID")

# ---------------------------- MONETARY -------------------------------------------------------------

df["Total_Spend"] = df["UnitPrice"] * df["Quantity"]
total_spend = df.groupby("CustomerID")["Total_Spend"].sum().reset_index()

average_transaction_value = total_spend.merge(total_transactions, on="CustomerID")
average_transaction_value["Average_Transaction_Value"] = average_transaction_value["Total_Spend"] / average_transaction_value["Total_Transactions"]

customer_data = pd.merge(customer_data, total_spend, on="CustomerID")
customer_data = pd.merge(customer_data, average_transaction_value[["CustomerID", "Average_Transaction_Value"]], on="CustomerID")

customer_data.head()

# ------------------------- FILTERING --------------------------------------------------------------

customer_data = customer_data[customer_data["Days_Since_Last_Purchase"] < 55]
customer_data = customer_data[(customer_data["Total_Spend"] > 10) & (customer_data["Total_Spend"] < 2000)]
customer_data = customer_data[customer_data["Total_Transactions"] > 3]

customers = customer_data["CustomerID"].to_list()

df = df[df["CustomerID"].isin(customers)]

In [56]:
c = df.copy()
count_purchases = c.groupby("CustomerID")["Total_Spend"].count().reset_index()
top_customers = count_purchases[count_purchases["Total_Spend"] > 50]

c = c[c["CustomerID"].isin(top_customers["CustomerID"])]


3075

In [57]:
unique_items = set(df["StockCode"])
len(unique_items)

3096

In [9]:
# to timestamp
df_copy = df.copy()

df_copy["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Timestamp"] = df_copy["InvoiceDate"].astype(np.int64) // 10 ** 9

df = df.sort_values(by="Timestamp")
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Transaction_Status,InvoiceDay,Total_Spend,Timestamp
308,536401,22110,BIRD HOUSE HOT WATER BOTTLE,1,2010-12-01 11:21:00,2.55,15862.0,United Kingdom,Completed,2010-12-01,2.55,1291202460
311,536401,22766,PHOTO FRAME CORNICE,1,2010-12-01 11:21:00,2.95,15862.0,United Kingdom,Completed,2010-12-01,2.95,1291202460
312,536401,22451,SILK PURSE BABUSHKA RED,1,2010-12-01 11:21:00,3.35,15862.0,United Kingdom,Completed,2010-12-01,3.35,1291202460
313,536401,22549,PICTURE DOMINOES,1,2010-12-01 11:21:00,1.45,15862.0,United Kingdom,Completed,2010-12-01,1.45,1291202460
314,536401,84744,S/6 SEW ON CROCHET FLOWERS,1,2010-12-01 11:21:00,1.25,15862.0,United Kingdom,Completed,2010-12-01,1.25,1291202460
...,...,...,...,...,...,...,...,...,...,...,...,...
391180,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,Completed,2011-12-09,16.60,1323435000
391181,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,Completed,2011-12-09,16.60,1323435000
391174,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680.0,France,Completed,2011-12-09,15.00,1323435000
391177,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680.0,France,Completed,2011-12-09,16.60,1323435000


In [10]:
df.to_csv("./raw_data/prepared.csv")

In [9]:
# CREATE ITEM TABLE TO SQL

import psycopg2

def execute_many(conn, df, table):
    tuples = [tuple(x) for x in df.values]
    cols = ','.join(list(df.columns))

    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s, %%s)" % (table, cols)

    cursor = conn.cursor()

    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    cursor.close()


conn = psycopg2.connect("dbname=postgres_db user=postgres_user password=postgres_password", port=5430)

# reference to .py file -> fetch_img_url.py
# create table with images

item_images = pd.read_csv("./image_item.csv")
item_images = item_images[["item_id", "image_url"]]

items_to_sql = df[["StockCode", "Description"]]

items_to_sql = items_to_sql.rename(columns={"StockCode": "item_id", "Description": "description"})
items_to_sql = items_to_sql.drop_duplicates(subset=["item_id"])

merged = pd.merge(item_images, items_to_sql, on="item_id")
merged = merged[["item_id", "description", "image_url"]]

execute_many(conn, merged, "items")

In [11]:
# Prepare to atomic format
df = df[["CustomerID", "StockCode", "Timestamp"]]
df = df.rename(columns={"CustomerID": "user_id:token", "StockCode": "item_id:token", "Timestamp": "timestamp:float"})
df


,user_id:token,item_id:token,timestamp:float
308,15862.0,22110,1291202460
311,15862.0,22766,1291202460
312,15862.0,22451,1291202460
313,15862.0,22549,1291202460
314,15862.0,84744,1291202460
...,...,...,...
391180,12680.0,23254,1323435000
391181,12680.0,23255,1323435000
391174,12680.0,22730,1323435000
391177,12680.0,23256,1323435000


In [12]:
# writing to recbole atomic file
df.to_csv("dataset/data/data.inter", index=False, sep="\t")